<a href="https://colab.research.google.com/github/hasune613/hello-world/blob/main/LGBM_fetch_20newsgroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from sklearn.datasets import fetch_20newsgroups

In [12]:
train_data = fetch_20newsgroups(subset='train')
valid_data = fetch_20newsgroups(subset='test')

In [8]:
import pandas as pd

In [16]:
train = pd.DataFrame({'text':train_data['data'],'target':train_data['target']})
valid = pd.DataFrame({'text':valid_data['data'],'target':valid_data['target']})

In [17]:
train.head()

,text,target
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14


In [18]:
valid.head()

,text,target
0,From: v064mb9k@ubvmsd.cc.buffalo.edu (NEIL B. ...,7
1,From: Rick Miller <rick@ee.uwm.edu>\nSubject: ...,5
2,From: mathew <mathew@mantis.co.uk>\nSubject: R...,0
3,From: bakken@cs.arizona.edu (Dave Bakken)\nSub...,17
4,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,19


In [19]:
train.shape,valid.shape

((11314, 2), (7532, 2))

In [23]:
print(train.text.values[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [24]:
print(train.target.values[0])

7


In [26]:
train_data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [28]:
train_data.target_names[7]

'rec.autos'

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
vectorizer = TfidfVectorizer(max_features = 1000)
X_train = vectorizer.fit_transform(train.text)
X_valid = vectorizer.transform(valid.text)

In [44]:
X_train.shape, X_valid.shape

((11314, 1000), (7532, 1000))

In [47]:
import lightgbm as lgb

In [49]:
train_set = lgb.Dataset(X_train,train.target)
valid_set = lgb.Dataset(X_valid, valid.target)

In [51]:
params = {
    'objective' : 'multiclass', # objective：モデルの種類。多クラスならmulticlss
    'metric' : 'multi_logloss', # metric：評価指標。多クラスならmulti_logloss
    'num_class' : 20 # num_class：クラス数。今回は20
}

In [63]:
model = lgb.train(
    params = params,
    train_set = train_set,
    valid_sets = [train_set,valid_set],
    num_boost_round = 300,
    early_stopping_rounds = 10,
    verbose_eval = 20 # 20サイクル毎に学習過程が表示
)

Training until validation scores don't improve for 10 rounds.
[20]	training's multi_logloss: 0.836126	valid_1's multi_logloss: 1.50292
[40]	training's multi_logloss: 0.399019	valid_1's multi_logloss: 1.28377
[60]	training's multi_logloss: 0.205618	valid_1's multi_logloss: 1.1976
[80]	training's multi_logloss: 0.111011	valid_1's multi_logloss: 1.15871
[100]	training's multi_logloss: 0.0626625	valid_1's multi_logloss: 1.1467
Early stopping, best iteration is:
[104]	training's multi_logloss: 0.0560676	valid_1's multi_logloss: 1.14572


In [64]:
preds = model.predict(X_valid)
print(preds.shape)

(7532, 20)


In [65]:
preds = preds.argmax(axis=1)
print(preds) 
print('-'*10)
print(valid.target.values)

[ 4 12  0 ...  9  6 15]
----------
[ 7  5  0 ...  9  6 15]


In [66]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(valid.target.values, preds)
print(acc)

0.6643653744025492


In [71]:
MAX_FEATURES = 100

vectorizer = TfidfVectorizer(max_features = MAX_FEATURES)
X_train = vectorizer.fit_transform(train.text)
X_valid = vectorizer.transform(valid.text)

train_set = lgb.Dataset(X_train,train.target )
valid_set = lgb.Dataset(X_valid, valid.target)

model = lgb.train(
    params = params,
    train_set = train_set,
    valid_sets = [train_set, valid_set],
    num_boost_round = 300,
    early_stopping_rounds = 10,
    verbose_eval = 20
)

preds = model.predict(X_valid).argmax(axis = 1)
acc = accuracy_score(valid.target, preds)
print(MAX_FEATURES , acc)

Training until validation scores don't improve for 10 rounds.
[20]	training's multi_logloss: 1.68977	valid_1's multi_logloss: 2.55002
[40]	training's multi_logloss: 1.15406	valid_1's multi_logloss: 2.42176
[60]	training's multi_logloss: 0.824784	valid_1's multi_logloss: 2.36232
[80]	training's multi_logloss: 0.60725	valid_1's multi_logloss: 2.33416
[100]	training's multi_logloss: 0.456219	valid_1's multi_logloss: 2.32447
[120]	training's multi_logloss: 0.348736	valid_1's multi_logloss: 2.32183
Early stopping, best iteration is:
[122]	training's multi_logloss: 0.340001	valid_1's multi_logloss: 2.32155
100 0.28744025491237385
